In [58]:
##training
import os,pickle
import pandas as pd
from utils.DataProcess import Start
from utils.ClooingTowerOptimize import CT_opt

opt = CT_opt()
opt.df = Start('HL_8')

opt.train()
with open(os.path.join('model','regr.pkl'), 'wb') as f:
    pickle.dump(opt, f)

冷卻水塔 MAE: 14.52
冰水主機 MAE: 24.11
冷卻水泵 MAE: 2.73


In [3]:
# eval plot
import os,pickle
import numpy as np
import plotly.graph_objects as go
from tqdm import tqdm
from utils.DataProcess import Start
from utils.ClooingTowerOptimize import CT_opt
with open(os.path.join('model','regr.pkl'), 'rb') as p:
    opt = pickle.load(p)


vs = []
for c in tqdm(range(2000)):
    try:
        opt.predict(c)
        v,fig = opt.plot(21,29)
        vs.append(v)
    except:
        pass


fig = go.Figure()
fig.add_traces(go.Scatter(x=opt.df.Datetime[-1000:][::-1],y=opt.df.condenser_supply_temp[-1000:][::-1],mode='lines',name='Manual'))
fig.add_traces(go.Scatter(x=opt.df.Datetime[-1000:][::-1],y=np.array(vs),mode='lines',name='AI'))
fig.add_traces(go.Scatter(x=opt.df.Datetime[-1000:][::-1],y=opt.df.Wet_bulb_temp[-1000:][::-1],mode='lines',name='Wet bulb'))

100%|██████████| 2000/2000 [00:40<00:00, 49.65it/s]


In [4]:
from plotly.graph_objs.layout import YAxis,XAxis,Margin
layout = go.Layout(
    # title="{}".format(self.df_last.Datetime),
    yaxis=YAxis(
        title="<b> Approach </b>"
    ),
    yaxis2 = YAxis(
        title="<b> Wet bulb temp </b>",
        overlaying= 'y', 
        side= 'right',
    ),
    yaxis3 = YAxis(
        title="<b> Wet bulb temp </b>",
        overlaying= 'y', 
        side= 'right',
    ),
    xaxis=dict(
        title="<b> Cost (Kw)</b>"
    ),
)
fig =  go.Figure(layout=layout)
fig.add_traces(go.Scatter(x=opt.df.Datetime[-2000:][::-1],y=np.array(vs),name='AI',yaxis='y2',opacity=0.5))
fig.add_traces(go.Scatter(x=opt.df.Datetime[-2000:][::-1],y=opt.df.Wet_bulb_temp[-2000:][::-1],name='Wet bulb',yaxis='y2'))
# fig.add_traces(go.Scatter(x=opt.df.Datetime[-2000:][::-1],y=opt.df.CT_eff[-2000:][::-1],name='CT_eff',yaxis='y1'))
# fig.add_traces(go.Scatter(x=opt.df.Datetime[-2000:][::-1],y=opt.df.system_KPI[-2000:][::-1],name='KPI',yaxis='y3'))
fig.add_traces(go.Scatter(x=opt.df.Datetime[-2000:][::-1],y=opt.df.condenser_supply_temp[-2000:][::-1],name='Manual',yaxis='y2',opacity=0.5))




In [5]:
fig =  go.Figure(layout=layout)
fig.add_traces(go.Scatter(x=opt.df.Datetime[-2000:][::-1],y=np.array(vs-opt.df.system_KPI[-2000:][::-1]),name='KPI',yaxis='y1',opacity=0.5))
fig.add_traces(go.Scatter(x=opt.df.Datetime[-2000:][::-1],y=opt.df.Wet_bulb_temp[-2000:][::-1],name='Wet bulb',yaxis='y2'))

In [64]:
# eval plot
import os,pickle
import numpy as np
import plotly.graph_objects as go
from tqdm import tqdm
from utils.DataProcess import Start
from utils.ClooingTowerOptimize import CT_opt
with open(os.path.join('model','regr.pkl'), 'rb') as p:
    opt = pickle.load(p)


vs = []
for c in tqdm(range(1000)):
    try:
        opt.predict(c)
        v,fig = opt.plot(21,29)
        vs.append(v)
    except:
        pass


fig = go.Figure()
fig.add_traces(go.Scatter(x=opt.df.Datetime[-1000:][::-1],y=opt.df.condenser_supply_temp[-1000:][::-1],mode='lines',name='Manual'))
fig.add_traces(go.Scatter(x=opt.df.Datetime[-1000:][::-1],y=np.array(vs),mode='lines',name='AI'))
fig.add_traces(go.Scatter(x=opt.df.Datetime[-1000:][::-1],y=opt.df.Enthalpy[-1000:][::-1],mode='lines',name='Wet bulb'))

100%|██████████| 1000/1000 [00:18<00:00, 53.05it/s]


In [3]:
a = opt.predict(1)
a,b = opt.plot(21,29)

In [4]:
a,b = opt.plot(21,29)

In [5]:
opt.df.plot.scatter(y='CT_VFD_OUT',x='CT_eff')

In [6]:
b

In [ ]:
df.Enthalpy

In [76]:
import plotly.express as px
df = opt.df.copy()
df = df[df.Enthalpy>0]
px.scatter(
    data_frame=df,
    x='CT_eff',
    y='CT_Total_KW',
    color='Enthalpy'
)

In [109]:
import plotly.express as px
df = opt.df.copy()
df = df[df.Wet_bulb_temp>0]
px.scatter(
    data_frame=df,
    x='CT_VFD_OUT',
    y='Approach',
    color='Wet_bulb_temp'
)

In [78]:
import plotly.express as px
df = opt.df.copy()
df = df[(df.CT_VFD_OUT>39)&(df.CT_VFD_OUT<41)]
px.scatter(
    data_frame=df,
    x='Wet_bulb_temp',
    y='Approach',
    color='Wet_bulb_temp'
)

In [79]:
import plotly.express as px
df = opt.df.copy()
df = df[(df.CT_VFD_OUT>42)&(df.CT_VFD_OUT<44)]
px.scatter(
    data_frame=df,
    x='Wet_bulb_temp',
    y='Approach',
    color='Wet_bulb_temp'
)

In [ ]:
df.Enthalpy

In [105]:
import sklearn
import numpy as np
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split

def pp (df,Hz,col):
    df = df[(df[col]>Hz-1)&(df[col]<Hz+1)]
    df = df[df.Wet_bulb_temp>0]
    X  = np.array(df.Wet_bulb_temp).reshape(-1, 1)
    y = np.array(df.Approach)
    ## 將數據集拆成訓練集與測試集
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

    regr = sklearn.linear_model.TweedieRegressor(power=2, alpha=0, max_iter=10000)
    regr.fit(X_train,y=y_train)
    cv = sklearn.model_selection.KFold(n_splits=5,shuffle=True)
    y_cv = sklearn.model_selection.cross_val_predict(regr,X_test,y=y_test,cv=cv)
    mae = round(sklearn.metrics.mean_absolute_error(y_test,y_cv),2)
    print(mae)

    res = []
    RG = np.linspace(14,28,100)
    for i in RG:
        res.append(regr.predict(i.reshape(-1, 1)))
    res = np.array(res).flatten()

    fig = go.Figure()

    fig.add_trace(
        go.Scatter(x = RG, y=res, mode='lines',name=f'{Hz}hz')
        )

    fig.add_trace(
    go.Scatter(x = df.Wet_bulb_temp, y=df.Approach, mode='markers',opacity=0.2,name=f'{Hz}hz',marker_color='red')
    )
    return go.Scatter(x = RG, y=res, mode='lines',name=f'{Hz}hz'),go.Scatter(x = df.Wet_bulb_temp, y=df.Approach, mode='markers',opacity=0.2,name=f'{Hz}hz')

In [106]:
df8 = Start('HL_8').copy()

In [108]:
fig = go.Figure()
fig.add_traces(pp(df8,43,'CT_VFD_OUT')[0])
fig.add_traces(pp(df8,43,'CT_VFD_OUT')[1])
fig.add_traces(pp(df8,40,'CT_VFD_OUT')[0])
fig.add_traces(pp(df8,40,'CT_VFD_OUT')[1])
# fig.add_traces(pp(df8,30,'CT_VFD_OUT')[0])
# fig.add_traces(pp(df8,30,'CT_VFD_OUT')[1])
# fig.add_traces(pp(df8,25,'CT_VFD_OUT')[0])
# fig.add_traces(pp(df8,25,'CT_VFD_OUT')[1])
# t2 = pp(df14,35,'CT_VFD_OUT')


0.36
0.36
0.42
0.43
